# Modelagem preditiva da RMB (2018-2025)
Este notebook traduz o plano de modelagem aprovado para responder às cinco perguntas de negócio do TCC. Partimos da camada Gold/Silver consolidada no repositório, treinamos modelos supervisionados para estimar as taxas de internações por agravos hídricos (por 10 mil habitantes) e geramos artefatos que alimentam diretamente as páginas restantes do dashboard no Looker Studio. Cada seção explica o objetivo antes das execuções para manter o fluxo reproduzível em ambientes locais ou no Colab.

## 1. Configuração do ambiente e paths
Definimos dinamicamente o diretório raiz para funcionar tanto localmente quanto no Colab, importamos as bibliotecas usadas no restante do notebook e criamos constantes para as pastas `data/`, `gold/`, `silver/` e `dashboard/material_para_dashboard`.

In [1]:
import os
from datetime import datetime, timezone
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from IPython.display import display

from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    make_scorer,
)
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except ModuleNotFoundError:
    drive = None
    IN_COLAB = False

def resolve_repo_root() -> Path:
    env_root = os.getenv('REPO_ROOT') or os.getenv('DATA_ROOT')
    if env_root:
        candidate = Path(env_root).expanduser().resolve()
        if (candidate / 'data').exists():
            return candidate
    for candidate in [Path.cwd().resolve()] + list(Path.cwd().resolve().parents):
        if (candidate / 'data').exists() and (candidate / 'README.md').exists():
            return candidate
    if IN_COLAB:
        if drive is not None:
            drive.mount('/content/drive', force_remount=False)
        colab_repo = Path('/content/drive/MyDrive/projeto-final-curso-i2a2')
        if (colab_repo / 'data').exists():
            return colab_repo
    raise FileNotFoundError(
        'Não foi possível localizar o diretório raiz do projeto. Defina REPO_ROOT ou DATA_ROOT.'
    )

PROJECT_ROOT = resolve_repo_root()
DATA_DIR = PROJECT_ROOT / 'data'
SILVER_DIR = DATA_DIR / 'silver'
GOLD_DIR = DATA_DIR / 'gold'
CONFIG_DIR = PROJECT_ROOT / 'config'
DASHBOARD_EXPORT_DIR = PROJECT_ROOT / 'dashboard' / 'material_para_dashboard'
DASHBOARD_EXPORT_DIR.mkdir(parents=True, exist_ok=True)

print(f'Raiz do projeto: {PROJECT_ROOT}')
print(f'Dados Gold/Silver: {GOLD_DIR}')
print(f'Exports do dashboard: {DASHBOARD_EXPORT_DIR}')


Raiz do projeto: /home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2
Dados Gold/Silver: /home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/data/gold
Exports do dashboard: /home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/dashboard/material_para_dashboard


## 2. Leitura da camada Gold + fallbacks SNIS
Reaproveitamos o mesmo pipeline do notebook exploratório: carregamos `gold_features_ano`, aplicamos o fallback do SNIS para colunas faltantes de serviço e verificamos a cobertura anual para garantir consistência das séries 2018-2025.

In [3]:
pd.options.display.float_format = "{:.2f}".format
EXPECTED_YEARS = list(range(2018, 2026))

RMB_CFG = (
    pd.read_csv(CONFIG_DIR / "rmb_municipios.csv")
    .query("is_rmb == 1")
    .assign(
        cod_mun=lambda df: df["ibge_code"].astype(str).str.zfill(7),
        municipio_cfg=lambda df: df["name"].str.upper(),
    )
)
print(f"Municípios monitorados: {RMB_CFG['municipio_cfg'].tolist()}")

gold_path = GOLD_DIR / "gold_features_ano.parquet"
if not gold_path.exists():
    raise FileNotFoundError(f"Dataset Gold não encontrado em {gold_path}")

gold = pd.read_parquet(gold_path).copy()
gold["cod_mun"] = gold["cod_mun"].astype(str).str.zfill(7)
gold["municipio"] = gold["municipio"].str.upper()

snis_path = GOLD_DIR / "snis_rmb_indicadores_v2.parquet"
service_fallback_cols = [
    "idx_atend_agua_total",
    "idx_atend_agua_urbano",
    "idx_coleta_esgoto",
    "idx_tratamento_esgoto",
    "idx_hidrometracao",
    "idx_perdas_distribuicao",
    "idx_perdas_lineares",
    "idx_perdas_por_ligacao",
    "tarifa_media_agua",
]
if snis_path.exists():
    snis = pd.read_parquet(snis_path).copy()
    snis["cod_mun"] = snis["cod_mun"].astype(str).str.zfill(7)
    available_cols = [col for col in service_fallback_cols if col in snis.columns]
    snis_subset = snis[["cod_mun", "ano"] + available_cols]
    gold = gold.merge(snis_subset, on=["cod_mun", "ano"], how="left", suffixes=("", "_snis"))
    for col in available_cols:
        snis_col = f"{col}_snis"
        if snis_col in gold.columns:
            gold[col] = gold[col].fillna(gold[snis_col])
            gold.drop(columns=snis_col, inplace=True)
else:
    print(
        "⚠️ Arquivo snis_rmb_indicadores_v2.* não encontrado; seguindo com colunas já presentes em Gold."
    )

coverage = (
    gold.groupby("ano")["cod_mun"]
    .nunique()
    .reset_index(name="municipios_com_dado")
    .sort_values("ano")
)
coverage["municipios_esperados"] = RMB_CFG.shape[0]
coverage["percentual_cobertura"] = coverage["municipios_com_dado"] / coverage["municipios_esperados"]
print("\nCobertura por ano (municípios com dado vs esperado):")
display(coverage)


Municípios monitorados: ['BELÉM', 'ANANINDEUA', 'MARITUBA', 'BENEVIDES', 'SANTA BÁRBARA DO PARÁ', 'SANTA IZABEL DO PARÁ', 'CASTANHAL', 'BARCARENA']

Cobertura por ano (municípios com dado vs esperado):


,ano,municipios_com_dado,municipios_esperados,percentual_cobertura
0,2018,8,8,1.00
1,2019,8,8,1.00
2,2020,8,8,1.00
3,2021,8,8,1.00
4,2022,8,8,1.00
5,2023,8,8,1.00
6,2024,8,8,1.00
7,2025,8,8,1.00


## 3. Construção do `analysis_df` para métricas e ranking
Reutilizamos o mesmo preparo do notebook exploratório (preenchimentos, normalizações e score de priorização) para manter os drivers alinhados com o dashboard e reaproveitar colunas derivadas como `deficit_tratamento`, `perdas_excesso` e `chuva_total_mm_lag1`.

In [4]:
analysis_df = gold.copy().reset_index(drop=True)
analysis_df = analysis_df.merge(RMB_CFG[['cod_mun', 'municipio_cfg']], on='cod_mun', how='left')
analysis_df['municipio'] = analysis_df['municipio_cfg'].fillna(analysis_df['municipio']).str.upper()
analysis_df.drop(columns=['municipio_cfg'], inplace=True)
analysis_df['ano'] = analysis_df['ano'].astype(int)
analysis_df.sort_values(['cod_mun', 'ano'], inplace=True)
analysis_df.reset_index(drop=True, inplace=True)

columns_to_fill = [
    'idx_atend_agua_total',
    'idx_tratamento_esgoto',
    'idx_coleta_esgoto',
    'idx_perdas_distribuicao',
    'pct_conformes_global',
    'despesa_saude_pc',
    'pct_despesa_investimentos_saude',
    'chuva_total_mm',
    'temp_media_c',
    'populacao',
    'internacoes_hidricas_10k',
    'internacoes_total_10k',
]
for col in columns_to_fill:
    if col in analysis_df.columns:
        analysis_df[col] = analysis_df.groupby('cod_mun')[col].transform(lambda s: s.fillna(s.median()))
        analysis_df[col] = analysis_df[col].fillna(analysis_df[col].median())

percent_cols = [
    'idx_atend_agua_total',
    'idx_tratamento_esgoto',
    'idx_coleta_esgoto',
    'pct_conformes_global',
]
for col in percent_cols:
    if col in analysis_df.columns:
        analysis_df[col] = analysis_df[col].clip(lower=0, upper=100)

analysis_df['deficit_atendimento'] = (100 - analysis_df['idx_atend_agua_total']).clip(lower=0, upper=100)
analysis_df['deficit_tratamento'] = (100 - analysis_df['idx_tratamento_esgoto']).clip(lower=0, upper=100)
analysis_df['alerta_qualidade'] = (100 - analysis_df['pct_conformes_global']).clip(lower=0, upper=100)
analysis_df['perdas_excesso'] = analysis_df['idx_perdas_distribuicao'].clip(lower=0)
analysis_df['chuva_total_mm_lag1'] = analysis_df.groupby('cod_mun')['chuva_total_mm'].shift(1)

def minmax_norm(series: pd.Series) -> pd.Series:
    series = series.fillna(series.median())
    delta = series.max() - series.min()
    if delta == 0 or np.isclose(delta, 0):
        return pd.Series(0.0, index=series.index)
    return (series - series.min()) / delta

analysis_df['score_priorizacao'] = (
    0.40 * minmax_norm(analysis_df['internacoes_hidricas_10k'])
    + 0.20 * minmax_norm(analysis_df['deficit_tratamento'])
    + 0.15 * minmax_norm(analysis_df['alerta_qualidade'])
    + 0.15 * minmax_norm(analysis_df['deficit_atendimento'])
    + 0.10 * minmax_norm(analysis_df['perdas_excesso'])
)
analysis_df['prioridade_categoria'] = pd.cut(
    analysis_df['score_priorizacao'],
    bins=[-np.inf, 0.33, 0.66, np.inf],
    labels=['Estável', 'Atenção', 'Crítico'],
)
print(f'Registros disponíveis para modelagem: {analysis_df.shape[0]} linhas, {analysis_df.shape[1]} colunas')


Registros disponíveis para modelagem: 64 linhas, 63 colunas


/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/an

## 4. Dataset de modelagem (target + features)
Selecionamos variáveis acionáveis (cobertura, perdas, investimentos, clima e finanças) como preditores e mantemos `internacoes_hidricas_10k` como alvo para alinhar com a priorização utilizada no painel.

In [5]:
TARGET = 'internacoes_hidricas_10k'
FEATURE_CANDIDATES: List[str] = [
    'idx_atend_agua_total',
    'idx_tratamento_esgoto',
    'idx_coleta_esgoto',
    'pct_conformes_global',
    'idx_perdas_distribuicao',
    'deficit_atendimento',
    'deficit_tratamento',
    'alerta_qualidade',
    'perdas_excesso',
    'chuva_total_mm',
    'chuva_total_mm_lag1',
    'temp_media_c',
    'umid_rel_media_pct',
    'despesa_saude_pc',
    'pct_despesa_investimentos_saude',
    'pct_despesa_medicamentos_saude',
    'pct_despesa_pessoal_saude',
    'pct_transferencias_sus_recursos',
    'pct_transferencias_sobre_despesa',
    'pct_receita_propria_asps',
    'populacao',
]
MODEL_FEATURES = [col for col in FEATURE_CANDIDATES if col in analysis_df.columns]
if not MODEL_FEATURES:
    raise ValueError('Nenhuma feature da lista está disponível no dataset Gold.')

model_df = analysis_df.dropna(subset=[TARGET]).copy()
print(f'Colunas usadas como features: {MODEL_FEATURES}')
print(f'Tamanho final da amostra: {model_df.shape[0]} linhas')


Colunas usadas como features: ['idx_atend_agua_total', 'idx_tratamento_esgoto', 'idx_coleta_esgoto', 'pct_conformes_global', 'idx_perdas_distribuicao', 'deficit_atendimento', 'deficit_tratamento', 'alerta_qualidade', 'perdas_excesso', 'chuva_total_mm', 'chuva_total_mm_lag1', 'temp_media_c', 'umid_rel_media_pct', 'despesa_saude_pc', 'pct_despesa_investimentos_saude', 'pct_despesa_medicamentos_saude', 'pct_despesa_pessoal_saude', 'pct_transferencias_sus_recursos', 'pct_transferencias_sobre_despesa', 'pct_receita_propria_asps', 'populacao']
Tamanho final da amostra: 64 linhas


## 5. Validação cruzada com GroupKFold (por município)
Treinamos Regressão Linear, Lasso e Random Forest usando pipelines com imputação + escala e avaliamos com GroupKFold (k=4) para evitar vazamento entre municípios. A métrica de seleção principal é o MAE.

In [6]:
def build_pipeline(estimator) -> Pipeline:
    numeric_transformer = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
        ]
    )
    preprocessor = ColumnTransformer(
        transformers=[('num', numeric_transformer, MODEL_FEATURES)],
        remainder='drop',
    )
    return Pipeline(steps=[('preprocess', preprocessor), ('model', estimator)])

MODELS = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(alpha=0.001, max_iter=10000),
    'RandomForest': RandomForestRegressor(
        n_estimators=600, random_state=42, min_samples_leaf=2, n_jobs=-1
    ),
}
def rmse_metric(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)
scoring = {
    'mae': 'neg_mean_absolute_error',
    'rmse': make_scorer(rmse_metric, greater_is_better=False),
    'r2': 'r2',
}
n_splits = min(4, model_df['cod_mun'].nunique())
gkf = GroupKFold(n_splits=n_splits)
cv_rows = []
for name, estimator in MODELS.items():
    pipeline = build_pipeline(estimator)
    cv_scores = cross_validate(
        pipeline,
        model_df[MODEL_FEATURES],
        model_df[TARGET],
        groups=model_df['cod_mun'],
        cv=gkf,
        scoring=scoring,
        n_jobs=-1,
    )
    cv_rows.append(
        {
            'model': name,
            'mae_mean': -cv_scores['test_mae'].mean(),
            'rmse_mean': -cv_scores['test_rmse'].mean(),
            'r2_mean': cv_scores['test_r2'].mean(),
        }
    )
cv_results = pd.DataFrame(cv_rows).sort_values('mae_mean')
display(cv_results)
best_model_name = cv_results.iloc[0]['model']
print(f'Modelo selecionado para holdout: {best_model_name}')


/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warn

,model,mae_mean,rmse_mean,r2_mean
2,RandomForest,4.77,6.30,-0.12
1,Lasso,469.99,708.31,-18845.45
0,LinearRegression,473.83,713.94,-19122.47


Modelo selecionado para holdout: RandomForest


## 6. Holdout 2025, previsões e erros municipais
Reservamos 2025 como holdout temporal, avaliamos o desempenho final e depois ajustamos o melhor modelo em todos os anos para gerar as previsões que alimentarão o dashboard.

In [8]:
HOLDOUT_YEAR = 2025
train_df = model_df[model_df["ano"] < HOLDOUT_YEAR].copy()
test_df = model_df[model_df["ano"] == HOLDOUT_YEAR].copy()
if test_df.empty:
    raise ValueError("Não há dados para o ano de holdout definido.")

best_estimator = clone(MODELS[best_model_name])
holdout_pipeline = build_pipeline(best_estimator)
holdout_pipeline.fit(train_df[MODEL_FEATURES], train_df[TARGET])
test_pred = holdout_pipeline.predict(test_df[MODEL_FEATURES])
holdout_metrics = {
    "model": best_model_name,
    "dataset": f"Holdout_{HOLDOUT_YEAR}",
    "mae": mean_absolute_error(test_df[TARGET], test_pred),
    "rmse": mean_squared_error(test_df[TARGET], test_pred, squared=False),
    "r2": r2_score(test_df[TARGET], test_pred),
}
display(pd.DataFrame([holdout_metrics]))

test_predictions = test_df[
    ["cod_mun", "municipio", "ano", "populacao", "prioridade_categoria", TARGET]
].copy()
test_predictions["y_pred"] = test_pred
test_predictions["erro_absoluto"] = (test_predictions["y_pred"] - test_predictions[TARGET]).abs()
test_predictions["ape_pct"] = np.where(
    test_predictions[TARGET].abs() > 0,
    (test_predictions["erro_absoluto"] / test_predictions[TARGET].abs()) * 100,
    np.nan,
)
print("\nErros por município no holdout:")
display(test_predictions[["municipio", "ano", TARGET, "y_pred", "erro_absoluto", "ape_pct"]])

municipio_error = (
    test_predictions.groupby("municipio")
    .agg(mae=("erro_absoluto", "mean"), ape_pct=("ape_pct", "mean"))
    .reset_index()
    .sort_values("mae")
)
display(municipio_error)

final_pipeline = build_pipeline(clone(MODELS[best_model_name]))
final_pipeline.fit(model_df[MODEL_FEATURES], model_df[TARGET])
model_df["pred_all"] = final_pipeline.predict(model_df[MODEL_FEATURES])


/home/anunnaki/Documentos/I2A2/tarefas/projeto_final_TCC/projeto-final-curso-i2a2/.venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


,model,dataset,mae,rmse,r2
0,RandomForest,Holdout_2025,4.18,5.46,-4.92



Erros por município no holdout:


,municipio,ano,internacoes_hidricas_10k,y_pred,erro_absoluto,ape_pct
7,ANANINDEUA,2025,2.22,10.43,8.21,369.92
15,BARCARENA,2025,4.39,5.84,1.46,33.24
23,BELÉM,2025,4.33,14.80,10.47,241.76
31,BENEVIDES,2025,0.73,4.24,3.52,485.36
39,CASTANHAL,2025,6.22,5.92,0.29,4.70
47,MARITUBA,2025,1.00,7.54,6.54,650.85
55,SANTA BÁRBARA DO PARÁ,2025,0.89,2.35,1.46,164.25
63,SANTA IZABEL DO PARÁ,2025,6.57,8.05,1.48,22.56


,municipio,mae,ape_pct
4,CASTANHAL,0.29,4.70
6,SANTA BÁRBARA DO PARÁ,1.46,164.25
1,BARCARENA,1.46,33.24
7,SANTA IZABEL DO PARÁ,1.48,22.56
3,BENEVIDES,3.52,485.36
5,MARITUBA,6.54,650.85
0,ANANINDEUA,8.21,369.92
2,BELÉM,10.47,241.76


## 7. Importâncias, permutação e elasticidades
Com o modelo final em mãos, calculamos importâncias diretas, importância por permutação no holdout e um quadro de elasticidades simulando melhorias incrementais nas variáveis acionáveis para 2025.

In [10]:
perm_result = permutation_importance(
    holdout_pipeline,
    test_df[MODEL_FEATURES],
    test_df[TARGET],
    n_repeats=50,
    random_state=42,
    scoring="neg_mean_absolute_error",
)
perm_importance_series = pd.Series(
    np.abs(perm_result.importances_mean), index=MODEL_FEATURES
).sort_values(ascending=False)

model_step = final_pipeline.named_steps["model"]
if hasattr(model_step, "feature_importances_"):
    feature_importances = pd.Series(
        model_step.feature_importances_, index=MODEL_FEATURES
    ).sort_values(ascending=False)
elif hasattr(model_step, "coef_"):
    coefs = np.ravel(model_step.coef_)
    feature_importances = pd.Series(np.abs(coefs), index=MODEL_FEATURES).sort_values(ascending=False)
else:
    feature_importances = pd.Series(dtype=float)

print("Importância do modelo (ordem decrescente):")
display(feature_importances.head(15))
print("Importância por permutação no holdout:")
display(perm_importance_series.head(15))

scenario_base = analysis_df.loc[
    analysis_df["ano"] == HOLDOUT_YEAR, ["cod_mun", "municipio", "ano"] + MODEL_FEATURES
].copy()
scenario_base["baseline_pred"] = final_pipeline.predict(scenario_base[MODEL_FEATURES])
ACTIONABLE_STEPS: List[Tuple[str, float]] = [
    ("idx_atend_agua_total", 5.0),
    ("idx_tratamento_esgoto", 5.0),
    ("pct_conformes_global", 5.0),
    ("pct_despesa_investimentos_saude", 0.5),
    ("pct_receita_propria_asps", 0.5),
    ("pct_transferencias_sus_recursos", 0.5),
]
scenario_rows = []
for feature, delta in ACTIONABLE_STEPS:
    if feature not in scenario_base.columns:
        continue
    adjusted = scenario_base.copy()
    if feature.startswith(("idx_", "pct_", "deficit", "alerta", "perdas")):
        adjusted[feature] = (adjusted[feature] + delta).clip(lower=0, upper=100)
    else:
        adjusted[feature] = adjusted[feature] + delta
    preds_delta = final_pipeline.predict(adjusted[MODEL_FEATURES])
    impact = preds_delta - scenario_base["baseline_pred"]
    scenario_rows.append(
        {
            "feature": feature,
            "delta_aplicado": delta,
            "impacto_medio_taxa": impact.mean(),
            "impacto_min_taxa": impact.min(),
            "impacto_max_taxa": impact.max(),
        }
    )
scenario_summary = pd.DataFrame(scenario_rows)
print("\nElasticidades simuladas (taxa prevista por 10k hab.):")
display(scenario_summary)


Importância do modelo (ordem decrescente):


populacao                          0.29
pct_transferencias_sus_recursos    0.20
pct_despesa_pessoal_saude          0.07
despesa_saude_pc                   0.07
pct_receita_propria_asps           0.07
chuva_total_mm                     0.07
umid_rel_media_pct                 0.05
pct_despesa_medicamentos_saude     0.03
temp_media_c                       0.02
pct_despesa_investimentos_saude    0.02
chuva_total_mm_lag1                0.02
pct_transferencias_sobre_despesa   0.02
pct_conformes_global               0.01
idx_tratamento_esgoto              0.01
alerta_qualidade                   0.01
dtype: float64

Importância por permutação no holdout:


populacao                         0.73
despesa_saude_pc                  0.14
temp_media_c                      0.12
pct_conformes_global              0.09
alerta_qualidade                  0.07
umid_rel_media_pct                0.05
deficit_atendimento               0.01
chuva_total_mm                    0.01
chuva_total_mm_lag1               0.01
idx_atend_agua_total              0.01
idx_perdas_distribuicao           0.01
idx_tratamento_esgoto             0.00
deficit_tratamento                0.00
perdas_excesso                    0.00
pct_despesa_investimentos_saude   0.00
dtype: float64


Elasticidades simuladas (taxa prevista por 10k hab.):


,feature,delta_aplicado,impacto_medio_taxa,impacto_min_taxa,impacto_max_taxa
0,idx_atend_agua_total,5.00,0.11,-0.01,0.27
1,idx_tratamento_esgoto,5.00,0.12,0.00,0.16
2,pct_conformes_global,5.00,0.19,-0.03,0.64
3,pct_despesa_investimentos_saude,0.50,-0.01,-0.17,0.11
4,pct_receita_propria_asps,0.50,-0.00,-0.00,0.00
5,pct_transferencias_sus_recursos,0.50,0.00,0.00,0.00


## 8. Export dos artefatos para o dashboard
Salvamos métricas (CV + holdout), importâncias, previsões históricas e cenários em `dashboard/material_para_dashboard/` para uso direto nas páginas restantes do Looker.

In [11]:
export_timestamp = datetime.now(timezone.utc).isoformat()
cv_export = cv_results.assign(dataset='GroupKFold').rename(
    columns={'mae_mean': 'mae', 'rmse_mean': 'rmse', 'r2_mean': 'r2'}
)
metrics_export = pd.concat(
    [
        cv_export[['model', 'dataset', 'mae', 'rmse', 'r2']],
        pd.DataFrame([holdout_metrics]),
    ],
    ignore_index=True,
)
metrics_export['gerado_em'] = export_timestamp
feature_importance_export = pd.concat(
    [
        feature_importances.rename('valor').to_frame().assign(tipo='model_importance'),
        perm_importance_series.rename('valor').to_frame().assign(tipo='permutation_importance'),
    ]
)
feature_importance_export = feature_importance_export.reset_index().rename(columns={'index': 'feature'})
feature_importance_export['gerado_em'] = export_timestamp
predictions_export = model_df[[
    'cod_mun',
    'municipio',
    'ano',
    'populacao',
    'prioridade_categoria',
    TARGET,
    'pred_all',
]].rename(columns={TARGET: 'taxa_observada', 'pred_all': 'taxa_prevista'})
predictions_export['erro_absoluto'] = (predictions_export['taxa_prevista'] - predictions_export['taxa_observada']).abs()
predictions_export['gerado_em'] = export_timestamp
scenario_export = scenario_summary.assign(gerado_em=export_timestamp)

model_metrics_path = DASHBOARD_EXPORT_DIR / 'modelagem_metricas.csv'
feature_path = DASHBOARD_EXPORT_DIR / 'modelagem_importancias.csv'
predictions_path = DASHBOARD_EXPORT_DIR / 'modelagem_previsoes.csv'
scenario_path = DASHBOARD_EXPORT_DIR / 'modelagem_cenarios.csv'
metrics_export.to_csv(model_metrics_path, index=False)
feature_importance_export.to_csv(feature_path, index=False)
predictions_export.to_csv(predictions_path, index=False)
scenario_export.to_csv(scenario_path, index=False)
print('Arquivos atualizados:')
print(f'  • {model_metrics_path.relative_to(PROJECT_ROOT)}')
print(f'  • {feature_path.relative_to(PROJECT_ROOT)}')
print(f'  • {predictions_path.relative_to(PROJECT_ROOT)}')
print(f'  • {scenario_path.relative_to(PROJECT_ROOT)}')


Arquivos atualizados:
  • dashboard/material_para_dashboard/modelagem_metricas.csv
  • dashboard/material_para_dashboard/modelagem_importancias.csv
  • dashboard/material_para_dashboard/modelagem_previsoes.csv
  • dashboard/material_para_dashboard/modelagem_cenarios.csv
